In [ ]:
from typing import Any, Tuple
import gdsfactory as gf
from gdsfactory.components.bend_s import bend_s
from gdsfactory.cross_section import strip
from gdsfactory.types import CrossSectionSpec
from gdspy import Path
import numpy as np
from scipy.special import binom
from phidl import Device as Component
from picwriter.components import WaveguideTemplate


@gf.cell
def coupler_adiabatic2(
    length1: float = 20.0,
    length2: float = 50.0,
    length3: float = 30.0,
    wg_sep: float = 1.0,
    input_wg_sep: float = 3.0,
    output_wg_sep: float = 3.0,
    dw: float = 0.1,
    port: Tuple[int, int] = (0, 0),
    direction: str = "EAST",
    cross_section: CrossSectionSpec = strip,
    **kwargs):

    control_points_input_top = [
            (0, 0),
            (length1 / 2.0, 0),
            (length1 / 2.0, -input_wg_sep / 2.0 + wg_sep / 2.0),
            (length1, -input_wg_sep / 2.0 + wg_sep / 2.0),]

    control_points_input_bottom = [
            (0, -input_wg_sep),
            (length1 / 2.0, -input_wg_sep),
            (length1 / 2.0, -input_wg_sep / 2.0 - wg_sep / 2.0),
            (length1, -input_wg_sep / 2.0 - wg_sep / 2.0),]

    control_points_output_top = [
            (length1 + length2, -input_wg_sep / 2.0 + wg_sep / 2.0),
            (
                length1 + length2 + length3 / 2.0,
                -input_wg_sep / 2.0 + wg_sep / 2.0,
            ),
            (
                length1 + length2 + length3 / 2.0,
                -input_wg_sep / 2.0 + output_wg_sep / 2.0,
            ),
            (
                length1 + length2 + length3,
                -input_wg_sep / 2.0 + output_wg_sep / 2.0,
            ),]

    control_points_output_bottom = [
            (length1 + length2, -input_wg_sep / 2.0 - wg_sep / 2.0),
            (
                length1 + length2 + length3 / 2.0,
                -input_wg_sep / 2.0 - wg_sep / 2.0,
            ),
            (
                length1 + length2 + length3 / 2.0,
                -input_wg_sep / 2.0 - output_wg_sep / 2.0,
            ),
            (
                length1 + length2 + length3,
                -input_wg_sep / 2.0 - output_wg_sep / 2.0,
            ),]

    def bezier_curve(t, control_points):
        """Returns bezier coordinates.

        Args:
            t: 1D array of points varying between 0 and 1.
        """
        xs = 0.0
        ys = 0.0
        n = len(control_points) - 1
        for k in range(n + 1):
            ank = binom(n, k) * (1 - t) ** (n - k) * t**k
            xs += ank * control_points[k][0]
            ys += ank * control_points[k][1]

        return (xs, ys)

    def bezier_inp_top(t):
        return bezier_curve(t, control_points_input_top)
    
    def bezier_inp_bottom(t):
        return bezier_curve(t, control_points_input_bottom)

    def bezier_output_bottom(t):
        return bezier_curve(t, control_points_output_bottom)
    
    def bezier_output_top(t):
        return bezier_curve(t, control_points_output_top)

    c = Component()
    x = gf.get_cross_section(cross_section, **kwargs)

    wg_width = x.width

    top_input = Path(width=wg_width)
    top_input.parametric(bezier_inp_top, max_points=199, tolerance=0.00001, final_width=wg_width + dw / 2.0, layer=gf.get_layer(x.layer)[0], datatype=gf.get_layer(x.layer)[1])
    c.add(top_input)

    bot_input = Path(width=wg_width)
    bot_input.parametric(bezier_inp_bottom, max_points=199, tolerance=0.00001, final_width=wg_width - dw / 2.0, layer=gf.get_layer(x.layer)[0], datatype=gf.get_layer(x.layer)[1])
    c.add(bot_input)

    top_output = Path(width=wg_width)
    top_output.parametric(bezier_output_top, max_points=199, tolerance=0.00001, layer=gf.get_layer(x.layer)[0], datatype=gf.get_layer(x.layer)[1])
    c.add(top_output)

    bot_output = Path(width=wg_width)
    bot_output.parametric(bezier_output_bottom, max_points=199, tolerance=0.00001, layer=gf.get_layer(x.layer)[0], datatype=gf.get_layer(x.layer)[1])
    c.add(bot_output)


    try:
        top_input_cladding = Path(width=2 * x.cladding_offsets[0] + wg_width + dw / 2.0)
        bot_input_cladding = Path(width=2 * x.cladding_offsets[0] + wg_width - dw / 2.0)
        top_output_cladding = Path(width=2 * x.cladding_offsets[0] + wg_width)
        bot_output_cladding = Path(width=2 * x.cladding_offsets[0] + wg_width)
    except TypeError:
        top_input_cladding = Path(width=2 * 2 + wg_width + dw / 2.0)
        bot_input_cladding = Path(width=2 * 2 + wg_width - dw / 2.0)
        top_output_cladding = Path(width=2 * 2 + wg_width + wg_width)
        bot_output_cladding = Path(width=2 * 2 + wg_width + wg_width)

    try:
        if kwargs["cladding_layers"] is None:
            kwargs["cladding_layers"] = (111, 0)
    except KeyError:
        kwargs["cladding_layers"] = (111, 0)

    top_input_cladding.parametric(bezier_inp_top, max_points=199, tolerance=0.00001, layer=kwargs["cladding_layers"][0], datatype=kwargs["cladding_layers"][1])
    c.add(top_input_cladding)
    
    bot_input_cladding.parametric(bezier_inp_bottom, max_points=199, tolerance=0.00001, layer=111, datatype=0)
    c.add(bot_input_cladding)

    top_output_cladding.parametric(bezier_output_top, max_points=199, tolerance=0.00001, layer=111, datatype=0)
    c.add(top_output_cladding)

    bot_output_cladding.parametric(bezier_output_bottom, max_points=199, tolerance=0.00001, layer=111, datatype=0)
    c.add(bot_output_cladding)


    kwargs_top_taper = kwargs
    kwargs_bot_taper = kwargs

    try:
        update_offset_top = []
        update_offset_bot = []
        
        for offset in kwargs["cladding_offsets"]:
            update_offset_top.append(offset)
            update_offset_bot.append(offset)
        kwargs_top_taper["cladding_offsets"] = update_offset_top
        kwargs_bot_taper["cladding_offsets"] = update_offset_bot
    except KeyError:
        kwargs_top_taper["cladding_offsets"] = [top_input_cladding.w]
        kwargs_bot_taper["cladding_offsets"] = [bot_input_cladding.w]
    try:
        update_layer_top = []
        update_layer_bot = []
        for layer in kwargs["cladding_layers"]:
            update_layer_top.append(gf.get_layer(layer))
            update_layer_bot.append(gf.get_layer(layer))
        kwargs_top_taper["cladding_layers"] = update_layer_top
        kwargs_bot_taper["cladding_layers"] = update_layer_bot
    except KeyError:
        kwargs_top_taper["cladding_layers"] = [(111, 0)]
        kwargs_bot_taper["cladding_layers"] = [(111, 0)]
    # cladding_cross_section_top = gf.get_cross_section("cross_section", **kwargs_top_taper)
    # cladding_cross_section_bot = gf.get_cross_section("cross_section", **kwargs_bot_taper)

    taper_top = gf.components.taper2(length=length2, width1=wg_width + dw / 2.0, width2=wg_width, cross_section=cross_section, **kwargs_top_taper)

    taper_bottom = gf.components.taper2(length=length2, width1=wg_width - dw / 2.0, width2=wg_width, cross_section=cross_section, **kwargs_bot_taper)

    c = gf.read.from_phidl(c)
    
    c.add_port("o2", (0,0), wg_width, 180, layer='PORT', cross_section=x.copy(width=wg_width))
    c.add_port("o1", (0,-input_wg_sep), wg_width, 180, layer='PORT', cross_section=x.copy(width=wg_width))
    try:
        c.add_port("top_taper_connect", (length1, -input_wg_sep / 2.0 + wg_sep / 2.0), wg_width, 0, layer='PORT', cross_section=x.copy(width=2 * x.cladding_offsets[0] + wg_width))
        c.add_port("bottom_taper_connect", (length1, -input_wg_sep / 2.0 - wg_sep / 2.0), wg_width, 0, layer='PORT', cross_section=x.copy(width=2 * x.cladding_offsets[0] + wg_width))
    except TypeError:
        c.add_port("top_taper_connect", (length1, -input_wg_sep / 2.0 + wg_sep / 2.0), wg_width, 0, layer='PORT', cross_section=x.copy(width=2 * 2 + wg_width))
        c.add_port("bottom_taper_connect", (length1, -input_wg_sep / 2.0 - wg_sep / 2.0), wg_width, 0, layer='PORT', cross_section=x.copy(width=2 * 2 + wg_width))

    taper_top_ref = c << taper_top
    
    taper_bottom_ref = c << taper_bottom

    taper_top_ref.connect("o1", c.ports["top_taper_connect"])
    taper_bottom_ref.connect("o1", c.ports["bottom_taper_connect"])


    c.ports = {
        "o1": c.ports["o1"],
        "o2": c.ports["o2"],
    }

    c.add_port("o3", (
                length1 + length2 + length3,
                -input_wg_sep / 2.0 + output_wg_sep / 2.0,
            ), wg_width, 0, layer='PORT', cross_section=x.copy(width=wg_width))

    c.add_port("o4", (
                length1 + length2 + length3,
                -input_wg_sep / 2.0 - output_wg_sep / 2.0,
            ), wg_width, 0, layer='PORT', cross_section=x.copy(width=wg_width))
    
    c.absorb(taper_top_ref)
    c.absorb(taper_bottom_ref)

    c2 = gf.Component()

    c2 << gf.geometry.union(c, by_layer=True)

    if x.info:
        c2.info.update(x.info)

    if x.add_pins:
        c2 = x.add_pins(c2)

    if x.add_bbox:
        c2 = x.add_bbox(c2)

    c2.ports = c.ports

    # c2.show(show_ports=True)
    return c2
# x.cladding_offsets = [10]
coupler = coupler_adiabatic2(length3=20, cladding_offsets=[0.5])
coupler.show(show_ports=True)
# coupler.ports

In [ ]:
from gdsfactory.components import mzi, coupler_adiabatic
from gdsfactory.path import arc
import gdsfactory as gf
c2 = gf.Component()
c2 << coupler_adiabatic(length3=20, cladding_offset=0.5)

coupler_ref = c2 << coupler
coupler_ref.movey(destination=15)
# curve = arc(radius=((9*(((-3+1)/2)**2 + (5)**2))/(2*np.pi*5)), angle=50)
# c << curve.extrude("strip").mirror((0,0), (15,0))
# c << coupler_adiabatic(length1=10, length2=5, length3=5, cladding_offset=0)

c2.plot()
c2.show(show_ports=True, show_subports=True)
c2.ports
# c2.to_3d().show()

In [ ]:
# new coupler FDTD simulation

# basic ipython configuration (reload source code automatically and plots inline)
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import tidy3d as td
from tidy3d import web

import gdsfactory as gf
import gdsfactory.simulation.gtidy3d as gt
import gdsfactory.simulation as sim

s = gt.get_simulation(coupler, is_3d=False, plot_modes=True)

In [ ]:
coupler_original = gf.components.coupler_adiabatic(length3=20, cladding_offset=0.5)
# new coupler FDTD simulation

# basic ipython configuration (reload source code automatically and plots inline)
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import tidy3d as td
from tidy3d import web

import gdsfactory as gf
import gdsfactory.simulation.gtidy3d as gt
import gdsfactory.simulation as sim

c = gf.components.coupler_adiabatic(length3=20, cladding_offset=0.5)
s = gt.get_simulation(c, is_3d=False, plot_modes=True)

In [ ]:
c = gf.components.mmi1x2()
s = gt.get_simulation(c, is_3d=False, plot_modes=True, )